In [0]:
%run ../../_utils

Note: you may need to restart the kernel using dbutils.library.restartPython() to use updated packages.
  Using cached Unidecode-1.3.8-py3-none-any.whl (235 kB)
Note: you may need to restart the kernel using dbutils.library.restartPython() to use updated packages.


In [0]:
from pyspark.sql.functions import col, count


# Camada GOLD

Na camada gold, as limpezas e ajustes já foram feitos, então essa camada é responsável por aplicar regras de negócio, agregações e junções de dados que convirjam para analises.

Essa é uma tabela sumarizada analítica.

O objetivo dela é informar os meses em que tiveram mais pedidos

In [0]:
tb_name = "gold.olist_total_orders_month_year"
dataset_location = "olist_total_orders_dataset"
target_location = f"dbfs:/FileStore/delta/gold/brazilian_ecommerce/{dataset_location}"

## 1 - Data ingestion


In [0]:
df_orders = spark.read.table("gold.olist_orders") # leituira da delta table central, orders (ja da propria gold)


## 2 - preparation

In [0]:
df = df_orders.groupBy("order_purchase_year", "order_purchase_month").agg(
    count("order_id").alias("total_orders_by_month_year"),
).orderBy("order_purchase_year", "order_purchase_month")

In [0]:
display(df)

order_purchase_year,order_purchase_month,total_orders_by_month_year
2016,9,4
2016,10,326
2016,12,1
2017,1,807
2017,2,1788
2017,3,2697
2017,4,2411
2017,5,3737
2017,6,3277
2017,7,4068



## Saving data

In [0]:
save_dataframe(df, format_mode="delta", table_name=tb_name, target_location=target_location)

[LOG] Saving gold.olist_total_orders_month_year delta on dbfs:/FileStore/delta/gold/brazilian_ecommerce/olist_total_orders_dataset... OK!



## create delta table

TODO: implementar UPSERT

o upsert serve para não precisar reescrever todos os dados, mas aproveitar do Delta para fazer um MERGE, caso um registro antigo tenha uma nova versão e INSERT para os dados que são novos

In [0]:
create_table(table_name=tb_name, target_location=target_location)

[LOG] Creating delta table gold.olist_total_orders_month_year on dbfs:/FileStore/delta/gold/brazilian_ecommerce/olist_total_orders_dataset... OK!


In [0]:
# exit para fechar a execução
dbutils.notebook.exit("OK")

In [0]:
%sql

select * from gold.olist_total_orders_month_year

order_purchase_year,order_purchase_month,total_orders_by_month_year
2016,9,4
2016,10,326
2016,12,1
2017,1,807
2017,2,1788
2017,3,2697
2017,4,2411
2017,5,3737
2017,6,3277
2017,7,4068
